In [1]:
import pandas as pd
import graphistry

graphistry.register(api=1, protocol="https", server="hub.graphistry.com", key="4f55eede43c4020fbe604f4bb159737568e7e890c202fc943f2164dd1bdc4adf")

In [2]:
# load data
pd_data = pd.read_csv('./kg_final_with_temporal_data_and_validated_inconsistencies.txt', sep='\t')
pd_data = pd_data[['Subject', 'Predicate', 'Object']]

# filter positives only
neg_predicates = [
    'confers no resistance to antibiotic',
    'not upregulated by antibiotic',
    'no represses',
    'no activates',]

def _check_match(x, predicates):
    flag = False
    for predicate in predicates:
        if predicate in x:
            flag = True
    return flag

pd_pos_data = pd_data[~pd_data['Predicate'].apply(lambda x: _check_match(x, neg_predicates))]

# assign predicate category
def map_func(label):
    if 'confers resistance to antibiotic' in label:
        return 'confers resistance to antibiotic'
    elif 'upregulated by antibiotic' in label:
        return 'upregulated by antibiotic'
    elif 'represses' in label:
        return 'represses'
    elif 'activates' in label:
        return 'activates'
    elif 'has' in label:
        return 'has'
    elif 'is involved in' in label:
        return 'is involved in'
    elif 'is part of' in label:
        return 'is part of'
    elif 'targeted by' in label:
        return 'targeted by'
    else:
        raise ValueError('Invalid label: {}'.format(label))

pd_pos_data['Predicate Category'] = pd_pos_data['Predicate'].apply(lambda x: map_func(x))


/home/jyoun/.local/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
g = graphistry.bind(
    source='Subject',
    destination='Object',
    edge_label='Predicate',)

g.plot(pd_pos_data)
